# Задание по курсу «Дискретная оптимизация», МФТИ, весна 2017

## Задача 1-1
## Учимся переводить между decision/evaluation/search на примере Bin Packing

**Даны:** 
* $[w_1, \dots , w_k], w_i \in [0, 1]$ — веса грузов  
* $n_{\text{bins}}$ — количество корзин грузоподъемностью 1.

Предполагается, что функция `solve_bp_decision(weights, n_bins)` решает «распознавательный» (decision) вариант задачи bin packing. На вход ей подаётся список весов и число корзин. Функция возвращает булевский ответ на вопрос «можно ли заданные веса раскидать по не более чем `n_bins` контейнерам? 

Напишите содержимое функции `solve_bp_evaluation`, которая возвращает оптимальное число корзин (решает evaluation variant задачи bin packing). 

Затем напишите содержимое функции `solve_bp_search`, которая возвращает список номеров корзин, которые при каком-нибудь оптимальном распределении присваиваются весам из списка `weights` (корзины нумеруются с единицы). 

Каждая из следующих функций должна вызывать предыдущую не более чем полиномиальное число раз.

In [4]:
def solve_bp_decision(weights: list, n_bins: int) -> bool:
    def able_to_pack(weights: list, bin_capacities) -> bool:
        return weights == [] or any( 
            able_to_pack( weights[:-1], bin_capacities[:i] + [capacity - weights[-1]] + bin_capacities[(i + 1):] ) 
            for i, capacity in enumerate(bin_capacities) if capacity >= weights[-1] 
        )

    return able_to_pack( sorted(weights), [1.0] * n_bins )

# To test this, run:
# solve_bp_decision([0.8, 0.09, 0.4, 0.7], 2)
# solve_bp_decision([0.8, 0.09, 0.4, 0.7], 3)

In [5]:
def solve_bp_evaluation(weights: list) -> int:
#   Задача оценки.
#   Возвращаем минимальное число корзин.
#   Для решения используем бинпоиск по ответу.
    left = 0
    right = len(weights)
    while(right - left > 1):
        middle = left + (right - left) // 2 # Ищем середину
        if ( solve_bp_decision(weights, middle)): # Есть ли решение для middle корзин?
            right = middle # Да: сдвигаем верхнюю границу.
        else:
            left = middle # Нет: сдвигаем нижнюю границу.
    min_n_bins = right
    return min_n_bins

In [6]:
import numpy as np

def solve_bp_search(weights: list):
    
    # Ищем решение в виде списка корзин.
    
    def next(current):
    # Вспомогательная функция.
    # Ищем предмет с минимальным номером,большим заданного,
    # который ещё не лежит ни в одной корзине.
        current += 1
        while ((current < lenght)and(solution[current] > 0)):
            current += 1
        if (current == lenght):
            current = -1
        return current

    lenght = len(weights)
    solution = np.zeros(lenght) # Массив для номеров корзин в решении.
    num = solve_bp_evaluation(weights) # Количество корзин в решении.
    
    for i in range(0, num): # Поочередно заполняем корзины.
        busket = next(-1)
        # Кладем в корзину первый попавшийся предмет. Здесь можно брать любой предмет,
        # поскольку в итоге каждый предмет окажется в какой-нибудь корзине.
        solution[busket] = i + 1 
        pretend = next(busket) # Первый кандидат на попадание в новую корзину.
        
        if(pretend != -1):
            current_weight = weights[pretend] # Свободное место в текущей корзине.
            while (pretend != -1):
                if (current_weight + weights[pretend] <= 1): # Кандидат помещается?
                    reserve = weights[pretend] # Запоминаем вес кандидата.
                    weights[pretend] = 0
                    weights[busket] += reserve
                    # Помещаем кандидата в текущую корзину, "склеивая" его с её содержимым
                    # На месте кандидата оставляем ноль, он не повлияет на решение.
                    if (solve_bp_evaluation(weights) == num): # Количество корзин не поменялось?
                        solution[pretend] = i + 1 # Значит, кандидат становится частью корзины.
                    else:
                        weights[busket] -= reserve
                        weights[pretend] = reserve # Отменяем "склеивание".
                pretend = next(pretend) # Следующий кандидат.
                
    return solution